In [1]:
import sys
import os
import json

import numpy as np

from pprint import pprint
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

print(sys.version)
pprint(sys.path)

3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]
['',
 '/home/viktor/dev/kolonial/ENV/lib/python35.zip',
 '/home/viktor/dev/kolonial/ENV/lib/python3.5',
 '/home/viktor/dev/kolonial/ENV/lib/python3.5/plat-x86_64-linux-gnu',
 '/home/viktor/dev/kolonial/ENV/lib/python3.5/lib-dynload',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '/home/viktor/dev/kolonial/ENV/lib/python3.5/site-packages',
 '/home/viktor/dev/kolonial/ENV/lib/python3.5/site-packages/IPython/extensions',
 '/home/viktor/.ipython',
 '/home/viktor/dev/recept-scanning']


In [9]:
from recread.ocrresponses.lib import read_receipt_from_google_ocr_json, get_overlap_size, get_overlap_map,\
get_distinct_lists, sort_overlap_groups_by_x_axis
from recread.receipt.models import Receipt
from recread.ocrresponses.rotation import straighten_annotations

receipt_dir = os.path.join('..', 'data', 'receipt_responses')
file_name = 'kvittering-keiser.jpg.json'

with open(os.path.join(receipt_dir, file_name)) as receipt_file:
    receipt_json = json.load(receipt_file)

#receipt = read_receipt_from_google_ocr_json(receipt_json)

#text_annotations = receipt.text_annotations

#vars(receipt).keys()


In [14]:
from operator import itemgetter


file_name = 'nQsDUYw.jpg.json'

with open(os.path.join(receipt_dir, file_name)) as receipt_file:
    receipt_json = json.load(receipt_file)

text_annotations = receipt_json['textAnnotations']
straighten_annotations(text_annotations)

ta = text_annotations[64]



def get_overlap_map_2(annotations, height_factor=.6):
    result = [[] for _ in annotations]
    for i, a in enumerate(annotations):
        a_y_center = get_y_center(a['boundingPoly'])
        a_height = get_y_height(a['boundingPoly'])
        for j, b in enumerate(annotations):
            b_y_center = get_y_center(b['boundingPoly'])
            b_height = get_y_height(b['boundingPoly'])
            
            ab_dist_center = abs(a_y_center - b_y_center)
            ab_min_height = min(a_height, b_height)
            ab_max_height = max(a_height, b_height)
            
            ab_is_same_line = ab_dist_center < ab_min_height * height_factor and ab_max_height < ab_min_height * 5
            
            if ab_is_same_line:
                result[i].append(j)
    return result


def find_optimal_height_factor(annotations):
    height_factors = range(1, 17, 2)
    results = []
    
    for hf in height_factors:
        results.append((hf, len(get_distinct_lists(get_overlap_map_2(annotations, hf / 10)))))
        if len(results) > 1 and results[-1][1] >= results[-2][1]:
            print(results)
            return results[-1]
    print('Could not find optimal height factor')
    print(results)
    return results[-1]

def get_y_height(ocr_poly):
    return abs(ocr_poly['vertices'][0]['y'] - ocr_poly['vertices'][3]['y'])

def get_y_center(ocr_poly):
    return sum([p[1] for p in get_poly_from_ocr_poly(ocr_poly)]) / 4

def get_poly_from_ocr_poly(ocr_poly):
    return ((p['x'], p['y']) for p in ocr_poly['vertices'])
    
def get_sorted_lines_by_y_axis(overlaps, annotations):
    return sorted(overlaps, key=lambda x: get_y_center(annotations[x[0]]['boundingPoly']))
    
optimal_height_factor = find_optimal_height_factor(text_annotations)
print(optimal_height_factor)


overlap_map = get_overlap_map_2(text_annotations[:], optimal_height_factor[0] / 10)
overlap_map = sort_overlap_groups_by_x_axis(overlap_map, text_annotations)
overlap_map = get_sorted_lines_by_y_axis(overlap_map, text_annotations)
distinct_overlap_map = get_distinct_lists(overlap_map)

rec = Receipt(distinct_overlap_map, text_annotations)

print('Lines found: {}'.format(len(distinct_overlap_map)))
for i, line in enumerate(rec.token_lines):
    print(i, ''.join(line), distinct_overlap_map[i])


[(1, 84), (3, 65), (5, 41), (7, 35), (9, 55)]
(9, 55)
Lines found: 55
0 Оча [1]
1 Apent7-23(8-21) [2, 3, 4, 5, 6, 7, 8, 9, 10]
2 Telefon40001225 [11, 12, 13, 14, 15]
3 Butikk1802-2Prixøya [16, 17, 18, 19, 20, 21]
4 Kvittering831012016-01-1815:50 [22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
5 BANANX-TRAPAKKET10.43 [35, 36, 37, 38, 39, 32, 33, 34]
6 0.834kg12.50kr/kg [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
7 0.834kg12.50kr/kg6 [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 58]
8 0.834kg12.50kr/kg6. [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 58, 59]
9 0.834kg12.50kr/kg6.70 [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 58, 59, 60]
10 COOPKIDNEYBØNNER6.70 [50, 51, 58, 59, 60]
11 COOPKIDNEYBØNNER12.50kr/kg6.70 [50, 51, 44, 45, 46, 47, 48, 49, 58, 59, 60]
12 COOPKIDNEYBØNNER.50kr/kg6.70 [50, 51, 45, 46, 47, 48, 49, 58, 59, 60]
13 COOPKIDNEYBØNNER50/kg6.70 [50, 51, 46, 48, 49, 58, 59, 60]
14 COOPKIKERTER410G5.50 [52, 53, 54, 61, 62, 63]
15 COOPTØRKERULLER45.20 [55, 56, 64, 65, 66]
16 Breavrunding0.17 [57, 67,